In [2]:
%cd /content
!rm -rf KoreanStandardPronunciation
!git clone https://github.com/dhkang01/KoreanStandardPronunciation.git
%cd KoreanStandardPronunciation


/content
Cloning into 'KoreanStandardPronunciation'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 55 (delta 29), reused 17 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 179.29 KiB | 3.26 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/KoreanStandardPronunciation


In [3]:
!pip install -q "transformers>=4.38.0" "datasets>=2.18.0" "peft>=0.11.0" accelerate huggingface_hub evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from google.colab import drive
drive.mount('/content/drive')
save_dir = "/content/drive/MyDrive/models/kocharelectra-pron-lora-adapter"

import os
import json

Mounted at /content/drive


In [6]:
from datasets import load_dataset

dataset_id = "dhkang01/KMA_dataset"
raw_ds = load_dataset(dataset_id, split="train")

raw_ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/356 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/447115 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'input', 'output'],
    num_rows: 447115
})

In [37]:
ds_train_tmp = raw_ds.train_test_split(test_size=0.1, seed=42)
train_ds = ds_train_tmp["train"]
tmp_ds   = ds_train_tmp["test"]

ds_val_test = tmp_ds.train_test_split(test_size=0.5, seed=42)
val_ds = ds_val_test["test"]
test_ds = ds_val_test["train"]

# train/val/test -> 90/5/5

train_ds = train_ds.select(range(120000))
val_ds = val_ds.select(range(1000))
test_ds = test_ds.select(range(100))


Tokenizer 다운로드

In [8]:
from KoCharELECTRA.tokenization_kocharelectra import KoCharElectraTokenizer

model_name = "monologg/kocharelectra-small-discriminator"

tokenizer = KoCharElectraTokenizer.from_pretrained(model_name)
print(tokenizer.tokenize("가나다"))

tokenizer_config.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'KoCharElectraTokenizer'.


['가', '나', '다']


output vocab

In [9]:
from collections import OrderedDict

# tokenizer.vocab은 OrderedDict(토큰 → ID)
token_list = list(tokenizer.vocab.keys())

pron2id = OrderedDict()
for idx, tok in enumerate(token_list):
    pron2id[tok] = idx

id2pron = {v: k for k, v in pron2id.items()}

len(pron2id), list(list(pron2id.items())[:10])

(11568,
 [('[PAD]', 0),
  ('[UNK]', 1),
  ('[CLS]', 2),
  ('[SEP]', 3),
  ('[MASK]', 4),
  (' ', 5),
  ('이', 6),
  ('다', 7),
  ('는', 8),
  ('에', 9)])

전처리 함수 정의 및 적용

복수 발음 허용 X

In [10]:
import numpy as np

max_length = 128  # 필요에 따라 조절

def preprocess_example(example):
    text = example["input"]
    pron = example["output"]  # List[List[str]]

    # KoCharElectra는 char 단위 토큰 + [CLS], [SEP]
    encoding = tokenizer(
        text,
        truncation=True,
        max_length=max_length,
        padding="max_length",  # DataCollator 써도 되지만 여기서는 고정 길이로
        return_tensors=None,
    )

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]

    # Electra/KoCharElectra: 대체로 [CLS] + chars + [SEP]
    # => 실제 문자 수 = len(text)
    # => pron 길이와 len(text)가 맞는다고 가정 (안 맞는 샘플은 나중에 필터 가능)
    seq_len = sum(attention_mask)  # 실제 non-pad 길이
    # [CLS] at 0, [SEP] at seq_len-1, chars in 1..seq_len-2

    labels = np.full_like(input_ids, fill_value=-100)  # default ignore_index

    # 문자 수와 pron 길이 안 맞으면 그냥 전부 ignore(-100)로 두고 스킵되게 할 수도 있음
    # 여기선 일단 최소한으로만 체크
    n_chars = seq_len - 2  # CLS, SEP 제외

    if len(pron) != n_chars:
        # 불일치하는 경우: 전부 padding label로 두고, 나중에 이런 샘플 비율 보고 판단
        print(f"Warning: pron len {len(pron)} != n_chars {n_chars} for text: {text}")
        print(f"in the case, pron: {"".join([l[0] for l in pron])}")
        encoding["labels"] = labels.tolist()
        return encoding

    for i in range(len(pron)):
        cand_list = pron[i]
        if not cand_list:
            continue
        label_id = pron2id[cand_list[0]]               # 첫 후보를 gold label로 사용
        if label_id < 6:                               # 특수토큰, 띄어쓰기는 사용 X
            continue

        token_pos = 1 + i                      # 0: [CLS], 1.. : chars
        if token_pos < seq_len - 1:            # 마지막 [SEP] 전까지만
            labels[token_pos] = label_id

    encoding["labels"] = labels.tolist()
    return encoding

In [38]:
train_tokenized = train_ds.map(
    preprocess_example,
    remove_columns=train_ds.column_names,
)

val_tokenized = val_ds.map(
    preprocess_example,
    remove_columns=val_ds.column_names,
)

test_tokenized = test_ds.map(
    preprocess_example,
    remove_columns=val_ds.column_names,
)

# too long seq is out.

# train_tokenized[0]


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

in the case, pron: 미구기 세르비아의 인꿘 타나브 리유로 유고에 대하 눤조를 중단하기로 결정한 사시리나 유고 내 공화국까늬 경계선 변경을 반대하는 미구기나 유럽 꾹까드릐 강경한 성명이 대세르비아를 꿈꾸는 세르비아의 밀로세비치 대통녕에 대한 규타늬 표시라고


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

모델 로드, LoRA 적용

encoder에 LoRA적용
classifier에 LoRA적용X, 전부 trainable

char_embed 설정

char->vec(41dim)

emb wrapping

In [12]:
# 14개 기본 자음 (쌍자음은 여기에 포함 X)
BASE_CONSONANTS = ["ㄱ","ㄴ","ㄷ","ㄹ","ㅁ","ㅂ","ㅅ","ㅇ","ㅈ","ㅊ","ㅋ","ㅌ","ㅍ","ㅎ"]
CONSONANT2IDX = {c: i for i, c in enumerate(BASE_CONSONANTS)}

DOUBLE_CONSONANTS = {"ㄲ": "ㄱ", "ㄸ": "ㄷ", "ㅃ": "ㅂ", "ㅆ": "ㅅ", "ㅉ": "ㅈ"}

# 종성 복합 받침 분해 (기존과 동일 의미)
CODA_COMPOUND = {
    "ㄳ": ("ㄱ", "ㅅ"),
    "ㄵ": ("ㄴ", "ㅈ"),
    "ㄶ": ("ㄴ", "ㅎ"),
    "ㄺ": ("ㄹ", "ㄱ"),
    "ㄻ": ("ㄹ", "ㅁ"),
    "ㄼ": ("ㄹ", "ㅂ"),
    "ㄽ": ("ㄹ", "ㅅ"),
    "ㄾ": ("ㄹ", "ㅌ"),
    "ㄿ": ("ㄹ", "ㅍ"),
    "ㅀ": ("ㄹ", "ㅎ"),
    "ㅄ": ("ㅂ", "ㅅ"),
}

In [13]:
# 단모음 10개
BASE_VOWELS = ["ㅏ","ㅓ","ㅗ","ㅜ","ㅡ","ㅣ","ㅐ","ㅔ","ㅚ","ㅟ"]
VOWEL2IDX = {v: i for i, v in enumerate(BASE_VOWELS)}

# (기존 COMPOSED_VOWELS 활용: 야/여/요/유/ㅘ/ㅙ/ㅝ/ㅞ 등)
COMPOSED_VOWELS = {
    "ㅑ": ("ㅣ", "ㅏ"),
    "ㅒ": ("ㅣ", "ㅐ"),
    "ㅕ": ("ㅣ", "ㅓ"),
    "ㅖ": ("ㅣ", "ㅔ"),
    "ㅛ": ("ㅣ", "ㅗ"),
    "ㅠ": ("ㅣ", "ㅜ"),

    "ㅘ": ("ㅗ", "ㅏ"),
    "ㅙ": ("ㅗ", "ㅐ"),
    "ㅝ": ("ㅜ", "ㅓ"),
    "ㅞ": ("ㅜ", "ㅔ"),
    "ㅢ": ("ㅡ", "ㅣ")
}

In [14]:
ONSETS  = list("ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ")
NUCLEI  = list("ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ")
CODAS   = [""] + list("ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ")

def decompose(syllable):
    code = ord(syllable) - 0xAC00
    onset_idx = code // 588
    nucleus_idx = (code % 588) // 28
    coda_idx = code % 28

    onset = ONSETS[onset_idx]
    nucleus = NUCLEI[nucleus_idx]
    coda = CODAS[coda_idx] if coda_idx != 0 else None

    return onset, nucleus, coda

In [15]:
def consonant_base_and_double(jamo):
    """
    자음 jamo를 (base_consonant, is_double)로 변환.
    예: 'ㄲ' -> ('ㄱ', 1), 'ㄷ' -> ('ㄷ', 0)
    """
    if jamo in DOUBLE_CONSONANTS:
        return DOUBLE_CONSONANTS[jamo], 1
    return jamo, 0

def get_onset_feature_15(onset_jamo):
    """
    onset: 14개 기본자음 one-hot + double_flag 1 = 15D
    """
    base, is_double = consonant_base_and_double(onset_jamo)
    feat = np.zeros(15, dtype=np.float32)
    if base not in CONSONANT2IDX:
        raise ValueError(f"Unknown onset consonant: {onset_jamo}")
    feat[CONSONANT2IDX[base]] = 1.0
    feat[14] = float(is_double)   # 마지막 차원: 쌍자음 토글
    return feat

def get_coda_feature_16(coda_jamo):
    """
    coda: 14개 기본자음 few-hot + double_flag + no_coda_flag = 16D
    - 복합 받침(ㄳ 등)은 few-hot으로 두 자음 bit 모두 1
    - coda가 None이면: no_coda_flag = 1, 나머지 0
    """
    feat = np.zeros(16, dtype=np.float32)

    # 마지막 차원: no_coda_flag
    if coda_jamo is None:
        feat[15] = 1.0
        return feat

    # 복합 받침 처리
    def apply_single_coda(j):
        base, is_double = consonant_base_and_double(j)
        if base not in CONSONANT2IDX:
            raise ValueError(f"Unknown coda consonant: {j}")
        feat[CONSONANT2IDX[base]] = 1.0
        # double flag: 14번째 인덱스
        if is_double:
            feat[14] = 1.0

    if coda_jamo in CODA_COMPOUND:
        a, b = CODA_COMPOUND[coda_jamo]
        apply_single_coda(a)
        apply_single_coda(b)
    else:
        apply_single_coda(coda_jamo)

    # no_coda_flag는 0 (이미 default)
    return feat


In [16]:
def _vowel_fewhot_single(jamo):
    """
    단일 모음 jamo에 대한 10D one-hot.
    (BASE_VOWELS에 없는 경우 에러)
    """
    if jamo not in VOWEL2IDX:
        raise ValueError(f"Unknown base vowel: {jamo}")
    v = np.zeros(10, dtype=np.float32)
    v[VOWEL2IDX[jamo]] = 1.0
    return v

def get_vowel_feature_10(nucleus_jamo):
    """
    nucleus: 10D few-hot
    - BASE_VOWELS에 있으면 one-hot
    - COMPOSED_VOWELS에 있으면 구성요소 둘의 벡터를 OR (few-hot)
    """
    # 단모음으로 바로 있는 경우
    if nucleus_jamo in VOWEL2IDX:
        return _vowel_fewhot_single(nucleus_jamo)

    # 이중/복합 모음
    if nucleus_jamo in COMPOSED_VOWELS:
        a, b = COMPOSED_VOWELS[nucleus_jamo]
        va = get_vowel_feature_10(a)
        vb = get_vowel_feature_10(b)
        # few-hot: 두 벡터를 OR (0/1)
        v = va + vb
        v = np.clip(v, 0.0, 1.0)
        return v

    # 그 밖의 모음은 필요에 따라 매핑 추가 가능
    raise ValueError(f"Unknown nucleus vowel: {nucleus_jamo}")

In [17]:
def get_syllable_feature_41(syllable):
    onset, nucleus, coda = decompose(syllable)

    onset_feat  = get_onset_feature_15(onset)          # 15D
    nucleus_feat = get_vowel_feature_10(nucleus)       # 10D
    coda_feat   = get_coda_feature_16(coda)            # 16D

    return np.concatenate([onset_feat, nucleus_feat, coda_feat])  # 41D


char_embed = np.zeros((len(pron2id), 41), dtype=np.float32) # (11568, 41)
print(char_embed.shape)

for pron, idx in pron2id.items():
    if len(pron) != 1:
        continue
    if ord(pron) < ord('가'):
        continue
    if ord('힣') < ord(pron):
        continue
    char_embed[idx] = get_syllable_feature_41(pron)

(11568, 41)


In [18]:
import torch

char_embed = torch.from_numpy(char_embed)

In [19]:
onset_embed = np.zeros((len(ONSETS), 15), dtype = np.float32)
for i, onset in enumerate(ONSETS):
    onset_embed[i] = get_onset_feature_15(onset)
onset_embed = torch.from_numpy(onset_embed)

vowel_embed = np.zeros((len(NUCLEI), 10), dtype=np.float32)
for i, nucleus in enumerate(NUCLEI):
    vowel_embed[i] = get_vowel_feature_10(nucleus)
vowel_embed = torch.from_numpy(vowel_embed)

coda_embed = np.zeros((len(CODAS), 16), dtype=np.float32)
for i, coda in enumerate(CODAS):
    if coda == "":
        coda = None
    coda_embed[i] = get_coda_feature_16(coda)
coda_embed = torch.from_numpy(coda_embed)

NewEmb:
```
char  ->  OldEmb  ->  emb
    + 41dim -> 128dim +
```

41dim: 발음정보 고정 emb
128dim: 학습가능, Dense layer

In [20]:
import torch.nn as nn
import torch.nn.functional as F

class ElectraEmbeddingWithNew(nn.Module):
    def __init__(self, electra_embeddings, electra_embeddings_project, char_embed, adapt):
        super().__init__()
        self.old = electra_embeddings                  # 기존 ElectraEmbeddings
        self.proj = electra_embeddings_project
        self.pron_dim = char_embed.size(1)
        self.adapt = adapt

        # new embedding (11568X41), 학습하지 않음
        self.register_buffer("char_embed", char_embed, persistent=True)

        # new embedding → 256 projection
        self.new_up = nn.Linear(self.pron_dim, 256)
        nn.init.zeros_(self.new_up.weight)
        nn.init.zeros_(self.new_up.bias)

    def forward(
        self,
        input_ids=None,
        token_type_ids=None,
        position_ids=None,
        inputs_embeds=None,
        past_key_values_length=0,
    ):
      # HF ElectraEmbeddings의 원래 forward와 동일한 시그니처로 맞추고,
      # 내부에서 self.old(...) 를 그대로 호출하는 방식이 더 안전함.

        # ---- Electra projection ----
        old_emb_128 = self.old(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        old_emb_256 = self.proj(old_emb_128)

        # ---- new embedding 41 차원 ----
        new_emb_41 = self.char_embed[input_ids]     # (B, L, 41)
        new_emb_256 = self.new_up(new_emb_41)

        # ---- 256 차원에서 add ----
        if self.adapt == False:
            return old_emb_256, new_emb_256
        final_emb = old_emb_256 + new_emb_256
        return final_emb, new_emb_256


Concat wrapper 모듈 적용





In [21]:
class ElectraWithCharEmbedding(nn.Module):
    """
    - peft_model.base_model.electra 를 backbone으로 사용
    - electra.embeddings 를 ElectraEmbeddingWithNew 로 교체
    - forward에서는 encoder 통과 후:
      - sequence_output (encoder output)
      - embedding_output (입력 임베딩)
      을 함께 반환
    """
    def __init__(self, peft_model, char_embed, adapt=True):
        super().__init__()
        self.peft_model = peft_model              # PeftModelForTokenClassification

        # ElectraModel (LoRA 포함)
        electra = peft_model.base_model.electra

        # electra emb 교체
        new_emb = ElectraEmbeddingWithNew(
            electra.embeddings,
            electra.embeddings_project,
            char_embed,
            adapt=adapt
        )
        electra.embeddings = new_emb
        electra.embeddings_project = None

        # char_embed를 바깥에서 접근할 수 있게 노출
        self.char_embed = new_emb.char_embed      # (vocab_size, 41)
        self.config = peft_model.base_model.config
        self.hidden_size = self.config.hidden_size

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        **kwargs,
    ):
        # 0) define electra
        electra = self.peft_model.base_model.electra

        # 1) input embedding
        embedding_output, new_embedding_output = electra.embeddings(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
        )  # (B, L, embed_dim=hidden_size)

        # 2) attention mask / head mask
        extended_mask = electra.get_extended_attention_mask(
            attention_mask,
            input_shape=input_ids.shape,
            device=input_ids.device,
        )

        head_mask = electra.get_head_mask(
            None, electra.config.num_hidden_layers
        )

        # 3) encoder 출력 (LoRA 적용됨)
        encoder_outputs = electra.encoder(
            embedding_output,
            attention_mask=extended_mask,
            head_mask=head_mask,
            output_attentions=electra.config.output_attentions,
            output_hidden_states=electra.config.output_hidden_states,
            return_dict=True,
        )

        sequence_output = encoder_outputs.last_hidden_state  # (B, L, hidden_size)

        # 필요하면 encoder_outputs도 같이 넘길 수 있음
        return {
            "sequence_output": sequence_output,
            "embedding_output": new_embedding_output, # new emb(only pron info)
            # "encoder_outputs": encoder_outputs,
        }


In [22]:
class PronunciationRNNCell(nn.Module):
    """
    한 음절에 대해:
      - 입력: seq_state (N, 256), ctx_vec (N, 192 = 3*64)
      - 출력:
        - new_seq_state: (N, 256)   (tanh 된 상태)
        - emb_pron:      (N, 41)    (activation 없음)
        - is_end:        (N, 1)     (sigmoid)
    여기서 N = B * L (배치 내 전체 토큰 수)
    """
    def __init__(self, hidden_size_rnn: int = 256, ctx_dim: int = 192, out_embed_dim: int = 41):
        super().__init__()
        self.hidden_size_rnn = hidden_size_rnn   # 256
        self.ctx_dim = ctx_dim                  # 192
        self.out_embed_dim = out_embed_dim      # 41

        # 1개의 fc로 256 + 41 + 1차원 한 번에 뽑기
        self.fc = nn.Linear(hidden_size_rnn + ctx_dim,
                            hidden_size_rnn + out_embed_dim + 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, seq_state, ctx_vec):
        """
        seq_state: (N, 256)
        ctx_vec:   (N, 192)
        """
        x = torch.cat([seq_state, ctx_vec], dim=-1)  # (N, 256+192=448)
        out = self.fc(x)                             # (N, 256+41+1=298)

        h_part    = out[:, :self.hidden_size_rnn]                  # (N, 256)
        emb_part  = out[:, self.hidden_size_rnn:self.hidden_size_rnn+self.out_embed_dim]  # (N, 41)
        end_part  = out[:, -1:].contiguous()                       # (N, 1)

        new_seq_state = torch.tanh(h_part)         # seq_state는 항상 tanh를 거친 형태
        emb_pron      = emb_part                  # 발음 벡터는 activation 없음
        is_end        = self.sigmoid(end_part)    # EOS 확률

        return new_seq_state, emb_pron, is_end


In [47]:
class PronunciationTaggerRNN(nn.Module):
    """
    ElectraWithCharEmbedding을 backbone으로 사용하는 발음 tagger.

    - backbone:
        input_ids, attention_mask, token_type_ids ->
        sequence_output (B, L, 256)
        embedding_output (B, L, 256)  # 여기서는 사용하지 않아도 됨

    - 이 모델:
        1) Electra hidden(256)을 23dim으로 축소 (act+fc)
        2) 23dim + pron_feat(41dim)를 concat → 64dim syllable 벡터
        3) prev/cur/next 64dim을 이어붙여 ctx_t (192dim) 생성
           * prev = t-1 (mask면 t-2)
           * next = t+1 (mask면 t+2)
        4) ctx_t를 2-layer MLP(192→512→256, 마지막 tanh)를 통해 h0로 사용
        5) RNNCell(h0, ctx_t) 한 번 호출 → emb_pron_t(41dim) 얻기
        6) emb_pron_t와 char_embed[labels] 사이의 MSE loss 계산

    - char_embed: (vocab_size, 41) few-hot 발음 feature
    """
    def __init__(
        self,
        backbone,              # ElectraWithCharEmbedding
        num_labels: int,
        pron2id,
        ignore_index: int = -100,
        space_mask_id: int = None,  # 필요하면 공백 토큰 id 지정 가능 (옵션)
    ):
        super().__init__()
        self.backbone = backbone
        self.ignore_index = ignore_index
        self.num_labels = num_labels
        self.space_mask_id = space_mask_id

        self.config = backbone.config
        self.hidden_size_rnn = backbone.hidden_size     # 256

        H_enc = self.hidden_size_rnn        # 256
        H_pron = backbone.char_embed.size(1)  # 41
        self.H_enc = H_enc
        self.H_pron = H_pron

        # 256 → 23 축소용
        self.hidden_shrink = nn.Linear(H_enc, 23)
        self.hidden_shrink_act = nn.GELU()

        # syllable vec: 23 + 41 = 64
        self.syll_dim = 64

        # h0 초기화용 2-layer MLP: 192(3*64) → 512 → 256
        self.h0_fc1 = nn.Linear(3 * self.syll_dim, 512)
        self.h0_act1 = nn.GELU()
        self.h0_fc2 = nn.Linear(512, H_enc)    # 256
        # 마지막은 tanh로 state 안정화
        self.h0_act2 = nn.Tanh()

        # RNN cell: 256 state, 192 context, 41 output embed
        self.cell = PronunciationRNNCell(
            hidden_size_rnn=H_enc,
            ctx_dim=3 * self.syll_dim,
            out_embed_dim=H_pron,
        )

        # char_embed (vocab_size, 41)
        self.char_embed = backbone.char_embed

        self.register_buffer("onset_embed", onset_embed, persistent=True)
        self.register_buffer("vowel_embed", vowel_embed, persistent=True)
        self.register_buffer("coda_embed", coda_embed, persistent=True)
        self.pron2id = pron2id

    def _build_mask_for_ctx(self, input_ids, attention_mask, labels=None):
        """
        ctx용 mask 정의:
        - 발음에 쓰지 않는 위치: CLS/SEP/공백 등
        - pad는 attention_mask == 0으로 따로 빠진다고 가정
        여기서는 기본적으로:
            mask_ctx = (labels == ignore_index) & (attention_mask == 1)
        로 두고, 필요 시 space_mask_id로 더 엄격히 잡을 수 있음.
        """
        mask_ctx = (attention_mask == 1)

        if labels is not None:
            mask_ctx = mask_ctx & (labels == self.ignore_index)

        # space_mask_id가 주어졌다면, 해당 토큰도 명시적으로 mask에 포함
        if self.space_mask_id is not None:
            space_positions = (input_ids == self.space_mask_id)
            mask_ctx = mask_ctx | space_positions

        return mask_ctx  # True = mask 위치 (발음에 안 쓰는 위치)

    def _build_prev_next_idx(self, mask_ctx, valid_mask):
        """
        pad를 제외하면 mask_ctx가 두 번 연속 나오지 않는다는 가정 하에,
        prev index를:
            prev1 = t-1
            prev2 = t-2
            prev = prev1 if prev1 not mask else prev2
        로 정의하고, next도 대칭으로 정의.

        valid_mask는 attention_mask == 1 같은 것 (pad 제외).
        """
        B, L = mask_ctx.shape
        device = mask_ctx.device

        # 기본 index
        idx = torch.arange(L, device=device)

        # --- prev ---
        prev1 = (idx - 1).clamp(min=0)      # [0,0,1,2,...]
        prev2 = (idx - 2).clamp(min=0)      # [0,0,0,1,...]
        prev1 = prev1.unsqueeze(0).expand(B, -1)  # (B, L)
        prev2 = prev2.unsqueeze(0).expand(B, -1)

        prev1_is_mask = mask_ctx.gather(1, prev1)  # (B, L)
        prev_idx = torch.where(prev1_is_mask, prev2, prev1)  # (B, L)

        # pad 위치는 prev 자체를 자기 자신으로 두거나 0으로 둬도 상관없음
        # 여기서는 그냥 0으로 설정
        prev_idx = prev_idx * valid_mask.long()

        # --- next ---
        next1 = (idx + 1).clamp(max=L - 1)
        next2 = (idx + 2).clamp(max=L - 1)
        next1 = next1.unsqueeze(0).expand(B, -1)
        next2 = next2.unsqueeze(0).expand(B, -1)

        next1_is_mask = mask_ctx.gather(1, next1)
        next_idx = torch.where(next1_is_mask, next2, next1)

        next_idx = next_idx * valid_mask.long()

        return prev_idx, next_idx

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        **kwargs,
    ):
        # 1) backbone 출력 (ElectraWithCharEmbedding)
        backbone_outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        seq_out = backbone_outputs["sequence_output"]   # (B, L, 256)
        B, L, H_enc = seq_out.shape

        # 2) 발음 feature: char_embed에서 41dim 가져오기
        #    (B, L) -> (B, L, 41)
        char_embed = self.char_embed.to(input_ids.device)
        pron_feat = char_embed[input_ids]          # (B, L, 41)

        # 3) hidden 256 → 23 → syllable 64 = [23; 41]
        hidden_23 = self.hidden_shrink_act(self.hidden_shrink(seq_out))  # (B, L, 23)
        syll_vec = torch.cat([hidden_23, pron_feat], dim=-1)             # (B, L, 64)

        # 4) ctx용 mask/valid_mask 만들기
        #    - valid_mask: pad 제외 (attention_mask == 1)
        #    - mask_ctx: 발음에 안 쓰는 위치 (CLS/SEP/공백 등)
        valid_mask = (attention_mask == 1)          # (B, L) bool
        mask_ctx = self._build_mask_for_ctx(input_ids, attention_mask, labels)

        # 5) prev/next index (pad 제외, mask는 prevprev 규칙 사용)
        prev_idx, next_idx = self._build_prev_next_idx(mask_ctx, valid_mask)

        # 6) prev/cur/next 64dim 가져오기
        D = self.syll_dim  # 64
        # gather를 위해 index를 (B, L, 1) → (B, L, D)로 확장
        gather_prev = prev_idx.unsqueeze(-1).expand(-1, -1, D)
        gather_next = next_idx.unsqueeze(-1).expand(-1, -1, D)

        syll_prev = syll_vec.gather(1, gather_prev)   # (B, L, 64)
        syll_next = syll_vec.gather(1, gather_next)   # (B, L, 64)

        # mask 위치 자체에서는 prev/next를 0으로
        syll_prev = syll_prev.masked_fill(mask_ctx.unsqueeze(-1), 0.0)
        syll_next = syll_next.masked_fill(mask_ctx.unsqueeze(-1), 0.0)

        # 7) ctx_t = [prev; cur; next]  → (B, L, 192)
        ctx = torch.cat([syll_prev, syll_vec, syll_next], dim=-1)  # (B, L, 192)

        # 8) h0 초기화: ctx_t → 512 → 256 → tanh
        ctx_flat = ctx.view(B * L, -1)          # (N, 192)
        h0 = self.h0_act1(self.h0_fc1(ctx_flat))  # (N, 512)
        h0 = self.h0_act2(self.h0_fc2(h0))        # (N, 256), tanh

        # pad 위치는 state를 0으로 만들어도 됨
        valid_flat = valid_mask.view(B * L)
        h0 = h0 * valid_flat.unsqueeze(-1)       # (N, 256)

        # 9) RNN cell 한 번 (현재 설계에서는 step=1)
        seq_state, emb_pron_flat, is_end = self.cell(h0, ctx_flat)  # (N,256), (N,41), (N,1)

        # 10) (B, L, 41)로 reshape
        pred_embed = emb_pron_flat.view(B, L, self.H_pron)  # (B, L, 41)
        pred_embed = char_embed[input_ids] + 0.1 * pred_embed  # (B, L, 41)

        # ===========================================================
        # 11) Loss 계산 (MSE)
        # ===========================================================
        loss = None
        if labels is not None:
            # labels: (B, L), ignore_index = -100
            with torch.no_grad():
                labels_clamped = labels.clone()
                labels_clamped = labels_clamped.masked_fill(
                    labels_clamped == self.ignore_index, 0
                )
                char_embed = self.char_embed.to(labels_clamped.device)
                labels_clamped = labels_clamped.to(char_embed.device)

                target_embed = char_embed[labels_clamped]  # (B, L, 41)

            mse = (pred_embed - target_embed) ** 2         # (B, L, 41)
            mask = (labels != self.ignore_index).unsqueeze(-1)  # (B, L, 1)
            mse = mse * mask

            denom = mask.sum().clamp(min=1)   # 유효 토큰 수
            loss = mse.sum() / denom

        # ===========================================================
        # 12) 최종 return
        # ===========================================================
        return {
            "loss": loss,
            "logits": pred_embed,      # (B, L, 41)
            # "is_end": is_end.view(B, L, 1)  # 필요하면 꺼내서 쓰면 됨
        }


    def predict_chars(self, logits):
        """
        logits: (B, L, 41) - 예측된 발음 feature 벡터
        return: pred_ids (B, L)

        초성(15) / 중성(10) / 종성(16) 별 cosine similarity 합산
        """

        B, L, D = logits.shape
        pred_flat = logits.reshape(B * L, D)               # (N, 41)

        onset_embed = self.onset_embed.to(pred_flat.device)
        vowel_embed = self.vowel_embed.to(pred_flat.device)
        coda_embed = self.coda_embed.to(pred_flat.device)

        # -------------------------------
        # 1) 자소 단위 분리
        # -------------------------------
        pred_on  = pred_flat[:, :15]       # (N, 15)
        pred_vow = pred_flat[:, 15:25]     # (N, 10)
        pred_coda= pred_flat[:, 25:41]     # (N, 16)

        # -------------------------------
        # 2) cosine similarity 계산
        # -------------------------------
        sim_onset  = F.cosine_similarity(
            pred_on.unsqueeze(1),    # (N, 1, 15)
            onset_embed.unsqueeze(0),     # (1, V, 15)
            dim=-1
        )  # (N, V)
        pred_onset = sim_onset.argmax(dim=-1)

        sim_vowel = F.cosine_similarity(
            pred_vow.unsqueeze(1),   # (N, 1, 10)
            vowel_embed.unsqueeze(0),    # (1, V, 10)
            dim=-1
        )  # (N, V)
        pred_vowel = sim_vowel.argmax(dim=-1)

        sim_coda = F.cosine_similarity(
            pred_coda.unsqueeze(1),  # (N, 1, 16)
            coda_embed.unsqueeze(0),   # (1, V, 16)
            dim=-1
        )  # (N, V)
        pred_coda = sim_coda.argmax(dim=-1)

        onset_idx  = pred_onset.long()         # (N,)
        vowel_idx  = pred_vowel.long()         # (N,)
        coda_idx   = pred_coda.long()          # (N,)

        codepoints = 0xAC00 + (onset_idx * 21 + vowel_idx) * 28 + coda_idx  # (N,)
        codepoints_list = codepoints.detach().cpu().tolist()
        chars = [chr(int(cp)) for cp in codepoints_list]

        pred_ids_list = [
            pron2id[ch] if ch in pron2id else 0
            for ch in chars
        ]

        pred_ids_flat = torch.tensor(
            pred_ids_list,
            device=logits.device,
            dtype=torch.long,
        )  # (N,)

        pred_ids = pred_ids_flat.view(B, L)    # (B, L)

        return pred_ids



학습 진행

In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [25]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits_torch = torch.tensor(logits)
    predictions_torch = pron_model_rnn.predict_chars(logits_torch)
    predictions = predictions_torch.detach().cpu().numpy()

    # ignore_index = -100 제거 후 accuracy 계산
    mask = labels != -100
    y_true = labels[mask]
    y_pred = predictions[mask]

    if len(y_true) == 0:
        return {"accuracy": 0.0}

    result = accuracy_metric.compute(predictions=y_pred, references=y_true)
    return {"accuracy": result["accuracy"]}


In [48]:
from transformers import AutoModelForTokenClassification

num_labels = len(pron2id)

base_model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/kocharelectra-small-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "key", "value", "dense"]  # Electra의 attention/FFN 모듈 이름 기준
)

peft_model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()

# print(model)

trainable params: 3,415,344 || all params: 17,445,216 || trainable%: 19.5775


In [50]:
# 1) backbone 생성
backbone = ElectraWithCharEmbedding(peft_model=peft_model, char_embed=char_embed, adapt=False)

# 2) RNN 기반 tagger
pron_model_rnn = PronunciationTaggerRNN(
    backbone=backbone,
    num_labels=num_labels,
    pron2id = pron2id,
    ignore_index=-100,
)

In [53]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="kocharelectra-pron-lora",

    learning_rate=1e-3,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    num_train_epochs=1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,

    eval_strategy="steps",
    eval_steps=200,

    save_strategy="steps",
    save_steps=200,
    save_total_limit=3,

    logging_strategy='steps',
    logging_steps=20,

    fp16=True,
    report_to="none",   # wandb 등 안 쓸 거면 none
)

trainer = Trainer(
    model=pron_model_rnn,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [54]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

trainer.train()


Step,Training Loss,Validation Loss,Accuracy
200,0.046700,0.043470,0.974658
400,0.031100,0.025533,0.987022
600,0.018200,0.017133,0.988964
800,0.016100,0.013819,0.990599
1000,0.012800,0.012072,0.992234
1200,0.012400,0.010850,0.992745
1400,0.011100,0.009931,0.992847
1600,0.010800,0.009570,0.993358
1800,0.010300,0.009411,0.993664


TrainOutput(global_step=1875, training_loss=0.021433413100242615, metrics={'train_runtime': 419.1772, 'train_samples_per_second': 286.275, 'train_steps_per_second': 4.473, 'total_flos': 0.0, 'train_loss': 0.021433413100242615, 'epoch': 1.0})

In [31]:
trainer.save_model(save_dir)   # ✅ 모델 가중치 저장 (pytorch_model.bin)
tokenizer.save_pretrained(save_dir)

# 발음 vocab도 같이 저장
with open(os.path.join(save_dir, "pron_vocab.json"), "w", encoding="utf-8") as f:
    json.dump(pron2id, f, ensure_ascii=False, indent=2)


Inference

In [32]:
from safetensors.torch import load_file

# 1) pron_vocab 로드
with open(os.path.join(save_dir, "pron_vocab.json"), encoding="utf-8") as f:
    pron2id = json.load(f)
id2pron = {v: k for k, v in pron2id.items()}

# 3) safetensors 로드
state = load_file(os.path.join(save_dir, "model.safetensors"))
pron_model_rnn.load_state_dict(state, strict=False)
pron_model_rnn.to('cuda')

# 4) tokenizer 로드
tokenizer = KoCharElectraTokenizer.from_pretrained(save_dir)

In [33]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer, return_tensors="pt")

test_loader = DataLoader(
    test_tokenized,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)

valid_loader = DataLoader(
    val_tokenized,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)

device = next(pron_model_rnn.parameters()).device
pron_model_rnn.eval()

all_pred_ids = []

with torch.no_grad():
    for batch in valid_loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        output = pron_model_rnn.forward(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            token_type_ids=batch.get("token_type_ids", None),
        )  # (B, L)
        pred_ids = pron_model_rnn.predict_chars(output["logits"])

        # GPU → CPU → numpy
        pred_ids = pred_ids.cpu().numpy()
        masks = batch["attention_mask"].cpu().numpy()
        input_ids = batch["input_ids"].cpu()
        labels = batch["labels"].cpu()

        texts = tokenizer.batch_decode(
            input_ids,
            skip_special_tokens=True
        )

        B = input_ids.size(0)

        for i in range(B):
            seq_ids = pred_ids[i]
            mask = masks[i]
            text = texts[i]
            label = labels[i]
            valid = (label != -100)

            pred = [
                id2pron[int(idx)]
                for idx, m in zip(seq_ids.tolist(), valid.tolist())
                if m
            ]

            gold = []
            for idx, m in zip(label.tolist(), valid.tolist()):
                if not m:
                    continue
                if idx == -100:
                    continue
                gold.append(id2pron[int(idx)])

            print("TEXT:", text)
            print("PRED:", "".join(pred))
            print("GOLD:", "".join(gold))
            print("-" * 60)

TEXT: 월말 현재
PRED: 월말현재
GOLD: 월말현재
------------------------------------------------------------
TEXT: 이번엔
PRED: 이번렌
GOLD: 이버넨
------------------------------------------------------------
TEXT: 만여 점의 무기 중
PRED: 만려쩜릐무끼쭝
GOLD: 마녀저믜무기중
------------------------------------------------------------
TEXT: 장애인 관련 단체와 전문가들의 의견을 듣고
PRED: 짱애인관련딴체와쩐문까들릐의껸를듣꼬
GOLD: 장애인괄련단체와전문가드릐의겨늘듣꼬
------------------------------------------------------------
TEXT: 찹쌀풀을 섞는다
PRED: 찹쌀풀를섞는다
GOLD: 찹쌀푸를성는다
------------------------------------------------------------
TEXT: 년
PRED: 년
GOLD: 년
------------------------------------------------------------
TEXT: 실현 가능성이 불투명한 실정
PRED: 씰현까능썽이불투명한실쩡
GOLD: 실현가능썽이불투명한실쩡
------------------------------------------------------------
TEXT: 대한항공 서강윤 홍보부장은
PRED: 대한항꽁써깡윤홍뽀뿌짱은
GOLD: 대한항공서강윤홍보부장은
------------------------------------------------------------
TEXT: 넘겼다
PRED: 넒겯따
GOLD: 넘겯따
------------------------------------------------------------
TEXT: 는 내용이나
PRED: 는내용이나
GOLD: 는내용이나
---------

dummy model 평가

In [34]:
class DummyTextAsPronModel(nn.Module):
    """
    input_ids에서 각 토큰의 char_embed를 그대로 logits으로 내보내는 더미 모델.
    => predict_chars(logits)를 돌리면 '입력 텍스트 그대로'가 예측 발음이 됨.
    """
    def __init__(self, char_embed: torch.Tensor):
        super().__init__()
        # char_embed: (vocab_size, H_pron)
        self.register_buffer("char_embed", char_embed)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        **kwargs,
    ):
        """
        input_ids: (B, L)
        labels: (B, L), ignore_index=-100
        """
        # (B, L, H_pron) : 각 위치 토큰 id -> 그 토큰의 발음 feature
        logits = self.char_embed[input_ids]

        # loss는 dummy (학습 안 할 거라 상관 없음)
        loss = None
        if labels is not None:
            loss = torch.tensor(0.0, device=logits.device)

        return {
            "loss": loss,
            "logits": logits,
        }


In [35]:
dummy_model = DummyTextAsPronModel(char_embed=char_embed).to("cuda")

from transformers import Trainer

dummy_trainer = Trainer(
    model=dummy_model,
    args=training_args,      # 기존 training_args 재사용해도 됨 (train 안 할 거면 epoch 이런 건 무시)
    train_dataset=train_tokenized,   # 안 써도 되지만 형식 맞춰둠
    eval_dataset=val_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# baseline: "철자 그대로 발음"의 accuracy
dummy_results = dummy_trainer.evaluate()
print(dummy_results)


{'eval_loss': 0.0, 'eval_model_preparation_time': 0.0002, 'eval_accuracy': 0.7053954629061925, 'eval_runtime': 1.8616, 'eval_samples_per_second': 537.161, 'eval_steps_per_second': 33.841}


In [36]:
# from google.colab import runtime
# runtime.unassign()
